# Comparison using bias of monthly spatial means by altitude bounds

## Load required libraries

In [1]:
# To reload external files automatically (ex: utils)    
from pathlib import Path     
import numpy as np    
import pandas as pd     
import netCDF4 as nc  
from netCDF4 import Dataset   
import datetime as dt
import calendar as cld
import matplotlib.pyplot as plt
import xarray as xr
import proplot as plot  # New plot library (https://proplot.readthedocs.io/en/latest/)
from scipy import stats
import seaborn as sns
from seaborn import scatterplot
import matplotlib.cm as cm
import itertools
import os

## Set variables

In [2]:
ref = 'era5' # era5_land or esa_gf 
"""
We define the threshold used to deduce number of days with snow over land from snow cover fraction
threshold = '0' <=> 0%, threshold = '1' <=> 1%, threshold = '3' <=> 3% 
"""
threshold = '1' 

# Set elevation ranges list
bounds = ['0m_5000m','1000m_5000m','1000m_1500m', '1500m_2000m', '2000m_5000m']  

y_scale = 'y_scale_adapted'
# Set input data path
path_cmip_raw_data = '/home/hchaabani/Data/Snow/snc/cmip6/raw_data'
path_input_cmip6 = '/home/hchaabani/Data/Snow/snc/cmip6/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_era5 = '/home/hchaabani/Data/Snow/snc/Reanalysis/ERA5/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_era5_land = '/home/hchaabani/Data/Snow/snc/Reanalysis/ERA-LAND/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_SIM_35km = '/home/hchaabani/Data/Snow/snc/SIM_35km/Modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_esa_gf = '/home/hchaabani/Data/Snow/snc/Satellite/ESA_CCI/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold

# Set path of directories in which results will be stored
output_path = '/home/hchaabani/Hamid/PHD-mountain-climate/First_Part/Results/figures/comparison/period_comparison/histogram/comparison_by_alt_bounds/bias/v0/threshold_'+threshold+'/'+y_scale+'/ref_'+ref

# Set others useful variables                 
period = slice('1982-01-01', '2014-12-31')  
cmip6_labels = os.listdir(path_cmip_raw_data)
products_labels = ['esa_gf','era5_land','era5', 
                   'SIM_35km']+cmip6_labels

paths_inputs = {'esa_gf':path_input_esa_gf,'era5_land':path_input_era5_land,'era5':path_input_era5, 
                'SIM_35km':path_input_SIM_35km} | {prod:path_input_cmip6 for prod in cmip6_labels}

dic_data = {bound:{key:'data' for key in products_labels} for bound in bounds}   
months_labels = {11:'November', 12:'December', 1:'January',2:'February',3:'Mars',4:'April'}
x_mons_labels=['JAN', 'FEB','MAR', 'APR','NOV', 'DEC']
parameters = ['snc_monthly_value_15','ext_days_with_snow_15']
bounds_titles = ['alt >= 0m', 'alt > 1000m', '1000m-1500m', '1500m-2000m', 'alt > 2000m']

keys_dic_markers = { 'esa_gf':'o','era5_land':'s','era5':'s',
                    'SIM_35km':'$0$'} | {prod:'$'+str(iprod+1)+'$' for iprod,prod in enumerate(cmip6_labels)}

colors_markers = {'esa_gf':'cyan','era5_land':'limegreen',
                  'era5':'darkgreen','SIM_35km':'red',
                  'BCC-CSM2-HR':'gray','IPSL-CM6A-ATM-ICO-HR':'orange', 
                   'IPSL-CM6A-ATM-ICO-VHR':'yellow','IPSL-CM6A-ATM-HR':'brown',
                   'EC-Earth3P-HR':'blue','CNRM-CM6-1-HR':'pink'}

imonths = [11,12,1,2,3,4]

# Parameter to plot
par = 'ext_days_with_snow_15'
colors = []
for prod in products_labels:
    colors.append(colors_markers[prod])

## Importing and preparing data

In [3]:
for bound in bounds:   
    for prod in products_labels:    
        dic_data[bound][prod] = xr.open_dataset(paths_inputs[prod]+ '/where_alt_between_'+ bound + '/snc_monthly_'+prod+'_'+bound+'.nc').sel(time=period).load()    

<div class="alert alert-block alert-success"; background-color:red> There are some missing dates in the time dimension of some products, we will add these dates and assign them nan values in order to homogenize the timesteps in all products </div> 

In [4]:
"""
We detect missing dates by applying the difference function between a complete dataset and the dataset that is not
complete      
""" 
for bound in bounds:
    for key in products_labels: 
        missing_dates = set(dic_data[bound]['era5_land'].time.values).difference(set(dic_data[bound][key].time.values))
        missing_dates = list(missing_dates)   
    
        if len(missing_dates) !=0:
            # Then we convert the list of missing date to warray dataarray 
            missing_date_ds = xr.DataArray(missing_dates, dims=["time"], coords=[missing_dates])
        
            # We constract a dataaray of the whole dates period  
            full_dates = xr.concat([dic_data[bound][key].time, missing_date_ds], dim="time")
    
            # We reindex the original dataset and fill empty values with nan
            dic_data[bound][key] = dic_data[bound][key].reindex(time=full_dates, fill_value=np.nan).sortby("time")       

<div class="alert alert-block alert-success"; background-color:red> We keep just the dates where the information about spatial meanis available for all products, if in one date a product gives no information (nan), we should assign nan value to all other products in this specific timestemp. </div> 

In [5]:
for bound in bounds:
    dic_data[bound]['esa_gf'] = dic_data[bound]['esa_gf'].drop_duplicates(dim='time',keep='first')

In [6]:
"""
We keep only the monthly values which are available for all 
product, in order to compute the metrics over the sames periods
""" 
""" 
for bound in bounds:
    for par in [par]: 
        for date in dic_data[bound]['era5_land'].time.values:
            for prod in products_labels:
                if np.isnan(dic_data[bound][prod].loc[{'time':date}][par].values):
                    for prod in products_labels:  
                        dic_data[bound][prod][par].loc[{'time':date}] = np.nan 
    print('done for the bound: '+bound)
"""

" \nfor bound in bounds:\n    for par in [par]: \n        for date in dic_data[bound]['era5_land'].time.values:\n            for prod in products_labels:\n                if np.isnan(dic_data[bound][prod].loc[{'time':date}][par].values):\n                    for prod in products_labels:  \n                        dic_data[bound][prod][par].loc[{'time':date}] = np.nan \n    print('done for the bound: '+bound)\n"

## Bias computing

In [7]:
# We initialize the dictionnaries that will contains bias values 
bias_values = {bound:{prod:'prod' for prod in products_labels} for bound in bounds}

In [8]:
products_labels.remove(ref)
ref_data = {bound:'data' for bound in bounds}
for bound in bounds:
    ref_data[bound] = dic_data[bound][ref]
    dic_data[bound].pop(ref)

In [9]:
# Monthly averages
for bound in bounds:
    for prod in products_labels:      
        bias_values[bound][prod] = dic_data[bound][prod] - ref_data[bound]
        bias_values[bound][prod] = bias_values[bound][prod].groupby('time.month').mean(dim='time')
        
    print('Done for the bound: '+bound)

Done for the bound: 0m_5000m
Done for the bound: 1000m_5000m
Done for the bound: 1000m_1500m
Done for the bound: 1500m_2000m
Done for the bound: 2000m_5000m


## Plots

In [10]:
# We define a function for figures formating 
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

## Monthly average histogram

In [11]:
y_labels=[-31,-20,-10,0,10,20,31]
y_title = 'bias of number of days with snow'      
y_max = 31
y_min = -31 

alpha_titles = ['a','b','c','d','e']           

# Define the figure and each axis for the 3 rows and 1 columns    
fig, axs = plt.subplots(nrows=5,ncols=1,dpi=400) #   figsize=(11,8.5)
plt.subplots_adjust(wspace=0.2,hspace=0.46)  
set_size(14, 14) 
        
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten() 

for ibound,bound in enumerate(bounds):
    for iprod,prod in enumerate(products_labels):
        
        aux = pd.concat([bias_values[bound][prod][par].to_dataframe().reset_index().drop(labels='month', axis=1) for prod in products_labels], axis=1)     
        aux.loc[[0,1,2,3,10,11]].plot.bar(ax=axs[ibound], width=0.8, edgecolor='k', 
                     linewidth=1, color=colors, grid=False, capsize=4, 
                                          error_kw=dict(ecolor='black',elinewidth=1))

    axs[ibound].set_title('\n ('+alpha_titles[ibound]+')  '+bounds_titles[ibound], 
                             fontsize=22)
    axs[ibound].legend(labels ='')
    
    axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)
    axs[ibound].set_yticks(ticks = y_labels)
    axs[ibound].set_yticklabels(y_labels, fontsize=16, rotation='horizontal')
    axs[ibound].set_ylim(y_min, y_max)
    
    axs[ibound].set_xlabel('')
    axs[ibound].set_xticks(ticks = [0,1,2,3,4,5]) 
    axs[ibound].set_xticklabels(x_mons_labels, fontsize=16)  
        
    axs[ibound].grid(visible=True)
                
axs[2].set_ylabel(y_title, fontsize=23)
 
fig.legend(labels = products_labels, ncol=5, fontsize=15, bbox_to_anchor=(0.869, 0.96))
        
axs[4].set_xlabel('Months', fontsize=23)      
    
# Storing files in two formats pdf and png 
# plt.savefig(output_path + '/bias_spatial-temporal_histogram_averages_threshold_'+threshold+'_parameter_'+par+'.png')
# plt.savefig(output_path + '/bias_spatial-temporal_histogram_averages_threshold_'+threshold+'_parameter_'+par+'.pdf')

/tmp/ipykernel_30147/2225421208.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)
/tmp/ipykernel_30147/2225421208.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)
/tmp/ipykernel_30147/2225421208.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)
/tmp/ipykernel_30147/2225421208.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)
/tmp/ipykernel_30147/2225421208.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[ibound].set_yticklabels(labels=y_labels, fontsize=20)


Text(0.5, 0, 'Months')

/home/hchaabani/anaconda3/envs/first_part/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.canvas.print_figure(bytes_io, **kw)
/home/hchaabani/anaconda3/envs/first_part/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.canvas.print_figure(bytes_io, **kw)
/home/hchaabani/anaconda3/envs/first_part/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.canvas.print_figure(bytes_io, **kw)
/home/hchaabani/anaconda3/envs/first_part/li